In [1]:
### Import Packages ###
import os
import ast
import numpy as np
import math as math
import pandas as pd
import random as random

### Local Packages ###
from utils.Prediction import *
from utils.Selector import *
from utils.Auxiliary import *
from utils.Main import *

In [2]:
### Get Directory ###
cwd = os.getcwd()
ParentDirectory = os.path.abspath(os.path.join(cwd, ".."))

### DataType ###
DataTypeInput = "BostonHousing"

### Parameter Vector ###
ParameterVector = pd.read_csv(os.path.join(ParentDirectory, "Data", "ParameterVectors", "ParameterVector" + DataTypeInput + ".csv"))

# Simulation

In [5]:
SimulationConfigInput = {'DataFileInput': 'BostonHousing',
                        'Seed': 1,
                        'TestProportion': 0.2,
                        'CandidateProportion': 0.8,
                        'SelectorType': 'TreeEnsembleQBCFunction',
                        'ModelType': 'TreeFarmsFunction',
                        'UniqueErrorsInput': 0,
                        'n_estimators': 100,
                        'regularization': 0.01,
                        'rashomon_bound_adder': 0.01,
                        'Type': 'Classification'}

### Sequential Learning Process ###
SimulationResults = OneIterationFunction(SimulationConfigInput)

Iteration: 0
null
Finding Optimal Objective...
treefarms reported successful execution
training completed. Number of trees in the Rashomon set: 481
{
  "false": {
    "false": {
      "false": {
        "complexity": 0.009999999776482582,
        "loss": 0.02500000037252903,
        "name": "Y",
        "prediction": "False"
      },
      "feature": 12,
      "name": "LSTAT_1",
      "reference": 1.0,
      "relation": "==",
      "true": {
        "complexity": 0.009999999776482582,
        "loss": 0.03750000149011612,
        "name": "Y",
        "prediction": "True"
      },
      "type": "rational"
    },
    "feature": 4,
    "name": "NOX_2",
    "reference": 1.0,
    "relation": "==",
    "true": {
      "complexity": 0.009999999776482582,
      "loss": 0.012500000186264515,
      "name": "Y",
      "prediction": "False"
    },
    "type": "rational"
  },
  "feature": 11,
  "model_objective": 0.125,
  "name": "PTRATIO_3",
  "reference": 1.0,
  "relation": "==",
  "true": {
    "

In [8]:
print("SelectionHistory:" , len(SimulationResults["SelectionHistory"]))
print("ErrorVec:" , len(SimulationResults["ErrorVec"]))
print("AllErrorVec:" , len(SimulationResults["AllErrorVec"]))
print("UniqueModelsInRashomonSet:" , len(SimulationResults["RashomonCommitteeDict"]["UniqueModelsInRashomonSet"]))
print("AllModelsInRashomonSet:" , len(SimulationResults["RashomonCommitteeDict"]["AllModelsInRashomonSet"]))

SelectionHistory: 324
ErrorVec: 324
AllErrorVec: 324
UniqueModelsInRashomonSet: 324
AllModelsInRashomonSet: 324


---

In [ ]:
### Set Up ###
ErrorVecSimulation = []
HistoryVecSimulation = []

### Run Code ###
# for i in range(0,len(ParameterVector)):
# for i in range(0,3):

    ### Set Up ###
    print("ParameterVector Simulation Number: ", i)

    ### Parameter Vector ###
    # SimulationConfigInput = {"DataFileInput": ParameterVector.iloc[i]["Data"],
    #                         "Seed": int(ParameterVector.iloc[i]["Seed"]),
    #                         "TestProportion": float(ParameterVector.iloc[i]["TestProportion"]),
    #                         "CandidateProportion": float(ParameterVector.iloc[i]["CandidateProportion"]),
    #                         "SelectorType": str(ParameterVector.iloc[i]["SelectorType"]), 
    #                         "ModelType": str(ParameterVector.iloc[i]["ModelType"]), 
    #                         "UniqueErrorsInput": int(ParameterVector.iloc[i]["UniqueErrorsInput"]),
    #                         "n_estimators":int(ParameterVector.iloc[i]["n_estimators"]),
    #                         "regularization":float(ParameterVector.iloc[i]["regularization"]),
    #                         "rashomon_bound_adder":float(ParameterVector.iloc[i]["rashomon_bound_adder"]),
    #                         "Type":ParameterVector.iloc[i]["Type"]
    #                         }
    SimulationConfigInput = {'DataFileInput': 'BostonHousing',
                             'Seed': '1',
                             'TestProportion': '0.2',
                             'CandidateProportion': '0.8',
                             'SelectorType': 'TreeEnsembleQBCFunction',
                             'ModelType': 'TreeFarmsFunction',
                             'UniqueErrorsInput': '0',
                             'n_estimators': '100',
                             'regularization': '0.01',
                             'rashomon_bound_adder': '0.01',
                             'Type': 'Classification'}

    ### Sequential Learning Process ###
    SimulationResults = OneIterationFunction(SimulationConfigInput)

    ### Append ###
    ErrorVecSimulation.append(SimulationResults["ErrorVec"])
    HistoryVecSimulation.append(SimulationResults["SelectionHistory"])

ParameterVector Simulation Number:  0


TypeError: Cannot cast scalar from dtype('<U1') to dtype('int64') according to the rule 'safe'

In [ ]:
ErrorVecSimulation1 = ErrorVecSimulation[0]

In [ ]:
ErrorVecSimulation2 = ErrorVecSimulation[1]

In [ ]:
ErrorVecSimulation3 = ErrorVecSimulation[2]